In [2]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

/Users/nsheng/miniforge3/envs/mfe-preprogram/lib/python3.10/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [3]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

In [4]:
connection_string = 'sqlite:///data/data.db'
events_db = 'crypto_events_database'

In [5]:
tokens = pd.read_sql(f'''
SELECT 
    DISTINCT token
FROM ohlc
''', connection_string)

In [6]:
dates = pd.read_sql(f'''
SELECT
    MAX(ts) as last_date
FROM ohlc
''', connection_string)
last_date = last_dow(pd.to_datetime(dates["last_date"][0]), 0)
first_date = last_date - timedelta(days=14)

In [7]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [8]:
for _, token in tokens["token"].iteritems():
    print(f'running for {token} for date {first_date} to {last_date}', end='\r')
    res = pm.execute_notebook(
        'template.ipynb',
        f'{base_output_folder}/market_analysis_{token}_{last_date}.ipynb',
        parameters = {
            'from_date': f'{first_date}',
            'to_date': f'{last_date}',
            'token': token,
            'connection_string': connection_string,
            'events_db': events_db
        }
    )

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

In [9]:
nbs = sb.read_notebooks('outputs')

In [10]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'avg_return': scrap['prices'].data.hrly_return.mean(),
        'avg_volatility': scrap['prices'].data.volatility.mean(),
        'mdd': scrap['prices'].data.mdd.min(),
        'range_close': scrap['prices'].data.close[-1] - scrap['prices'].data.close[0]}) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,avg_return,avg_volatility,mdd,range_close
0,AAVE,-9.116028e-04,0.212356,-0.379501,-58.6450
1,ADA,-5.694307e-04,0.217477,-0.313607,-0.2445
2,ATOM,-4.556594e-04,0.299425,-0.383038,-2.7560
3,AVAX,-8.330764e-04,0.286536,-0.388197,-22.9300
4,BTC,-4.208531e-04,0.154894,-0.213690,-6999.8400
5,COMP,-1.174897e-03,0.224354,-0.384274,-88.2800
6,CRV,-8.845980e-04,0.328177,-0.404270,-1.0362
7,DOT,-6.433136e-04,0.245633,-0.356320,-6.0050
8,ETH,-1.395915e-04,0.174104,-0.190120,-141.7100
9,SOL,-5.063388e-04,0.242432,-0.321246,-26.6870
